In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from task1 import vanilla_model
from utils import load_data

tf.config.set_visible_devices([], 'GPU')

In [2]:
dataset = 'mitbih' # 'mitbih' or 'ptbdb'
X,Y,X_test,Y_test = load_data(dataset)

### Task 1

Solve both datasets with vanilla RNNs and CNNs.
- For the binary one, report accuracy, AUROC and AUPRC.
- For the non-binary one, report accuracy.
- Compare with baseline.

In [3]:
model_type = 'rnn'
model = vanilla_model(type=model_type,dataset=dataset)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 187, 1)]          0         
                                                                 
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 19,141
Trainable params: 19,141
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

if dataset =='mitbih':
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
elif dataset == 'ptbdb':
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
else:
    raise NotImplementedError('wrong dataset name')

early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0, patience = 5,verbose=2)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience =3, verbose = 1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='../chckpt',monitor='val_loss',verbose=1,save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='../logs/'+dataset+'/vanilla_lstm',write_images=True)

if dataset =='mitbih':
    metrics= ['accuracy']
elif dataset == 'ptbdb':
    auroc = tf.keras.metrics.AUC(curve='ROC',from_logits = True,name='auroc')
    auprc = tf.keras.metrics.AUC(curve='PR',from_logits = True,name='auprc')
    metrics= ['accuracy',auroc,auprc]
else:
    raise NotImplementedError('wrong dataset name')

model.compile(optimizer=optimizer, loss= loss, metrics = metrics)

model.fit(x=X,y=Y,batch_size=64,epochs=100,verbose=2,shuffle=True,callbacks=[early_stopper,lr_scheduler,checkpointer,tensorboard],validation_split = 0.1)

model.evaluate(x=X_test,y=Y_test)

### Task 4

option 1: Transfer learning with RNNs, frozen base model.

Train RNN model with MIT-BIH Arrhythmia Database.\
− Remove output layer(s) from model and feed it with the PTB Diagnostic ECG Database. The outputs are the
vector representation of the PTB Diagnostic ECG Database samples.\
− Take the representations of the samples of PTB Diagnostic ECG Database and train a new small
feedforward neural network equivalent to the layers you removed from the first model.

In [4]:
model.load_weights('../chckpt_lstm_mitbih')

#model.trainable = False # freeze the base model

x = model.layers[0].output
x = model.layers[1](x)
x = tf.keras.layers.Dense(units=128, activation='relu')(x)
x = tf.keras.layers.Dense(units=64, activation='relu')(x)
outputs = tf.keras.layers.Dense(units=1, activation=None)(x)

model2 = tf.keras.Model(inputs= model.layers[0].input, outputs = outputs)

for l in model2.layers:
    print(l.name, l.trainable)

input_1 False
lstm False
dense_2 True
dense_3 True
dense_4 True


In [5]:
dataset = 'ptbdb'
X,Y,X_test,Y_test = load_data(dataset)

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)


early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',min_delta=0, patience = 5,verbose=2)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience =3, verbose = 1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='../chckpt',monitor='val_accuracy',verbose=1,save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='../logs/'+dataset+'/task4',write_images=True)


auroc = tf.keras.metrics.AUC(curve='ROC',from_logits = True,name='auroc')
auprc = tf.keras.metrics.AUC(curve='PR',from_logits = True,name='auprc')
metrics= ['accuracy',auroc,auprc]


model2.compile(optimizer=optimizer, loss= loss, metrics = metrics)

model2.fit(x=X,y=Y,batch_size=64,epochs=100,verbose=2,shuffle=True,callbacks=[early_stopper,lr_scheduler,checkpointer,tensorboard],validation_split = 0.1)

model2.evaluate(x=X_test,y=Y_test)

Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.73047, saving model to ../chckpt


INFO:tensorflow:Assets written to: ../chckpt/assets


INFO:tensorflow:Assets written to: ../chckpt/assets


164/164 - 8s - loss: 0.4898 - accuracy: 0.7341 - auroc: 0.7691 - auprc: 0.8990 - val_loss: 0.4819 - val_accuracy: 0.7305 - val_auroc: 0.7950 - val_auprc: 0.9030 - lr: 0.0010 - 8s/epoch - 50ms/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.73047
164/164 - 4s - loss: 0.4602 - accuracy: 0.7611 - auroc: 0.8040 - auprc: 0.9164 - val_loss: 0.4749 - val_accuracy: 0.7305 - val_auroc: 0.8049 - val_auprc: 0.9084 - lr: 0.0010 - 4s/epoch - 26ms/step
Epoch 3/100

Epoch 3: val_accuracy improved from 0.73047 to 0.75536, saving model to ../chckpt


INFO:tensorflow:Assets written to: ../chckpt/assets


...

164/164 - 7s - loss: 0.3915 - accuracy: 0.8100 - auroc: 0.8686 - auprc: 0.9455 - val_loss: 0.4233 - val_accuracy: 0.7957 - val_auroc: 0.8590 - val_auprc: 0.9364 - lr: 2.0000e-04 - 7s/epoch - 40ms/step
Epoch 15/100

Epoch 15: val_accuracy did not improve from 0.79571
164/164 - 4s - loss: 0.3901 - accuracy: 0.8089 - auroc: 0.8698 - auprc: 0.9461 - val_loss: 0.4248 - val_accuracy: 0.7923 - val_auroc: 0.8580 - val_auprc: 0.9360 - lr: 2.0000e-04 - 4s/epoch - 25ms/step
Epoch 16/100

Epoch 16: val_accuracy improved from 0.79571 to 0.80086, saving model to ../chckpt


INFO:tensorflow:Assets written to: ../chckpt/assets


INFO:tensorflow:Assets written to: ../chckpt/assets


164/164 - 7s - loss: 0.3890 - accuracy: 0.8102 - auroc: 0.8701 - auprc: 0.9461 - val_loss: 0.4229 - val_accuracy: 0.8009 - val_auroc: 0.8624 - val_auprc: 0.9380 - lr: 2.0000e-04 - 7s/epoch - 40ms/step
Epoch 17/100

Epoch 17: val_accuracy did not improve from 0.80086
164/164 - 4s - loss: 0.3880 - accuracy: 0.8109 - auroc: 0.8711 - auprc: 0.9467 - val_loss: 0.4215 - val_accuracy: 0.7948 - val_auroc: 0.8600 - val_auprc: 0.9370 - lr: 2.0000e-04 - 4s/epoch - 27ms/step
Epoch 18/100

Epoch 18: val_accuracy did not improve from 0.80086
164/164 - 4s - loss: 0.3864 - accuracy: 0.8126 - auroc: 0.8726 - auprc: 0.9473 - val_loss: 0.4224 - val_accuracy: 0.7906 - val_auroc: 0.8591 - val_auprc: 0.9366 - lr: 2.0000e-04 - 4s/epoch - 22ms/step
Epoch 19/100

Epoch 19: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.

Epoch 19: val_accuracy did not improve from 0.80086
164/164 - 4s - loss: 0.3858 - accuracy: 0.8121 - auroc: 0.8734 - auprc: 0.9476 - val_loss: 0.4142 - val_accuracy: 0.7923

[0.41839462518692017,
 0.8024733662605286,
 0.8575666546821594,
 0.9357375502586365]